# Call the 2 algoritms Hungarian and Auction Vickrey

### Work for University Class - Sistemas Robóticos Cooperativos


MRobalinho

30-11-2020

In [1]:
import os
import numpy  as np
import pandas as pd
import random
import datetime
from datetime import date

In [2]:
'''
# pip install Munkres
Munkres implementation for Python
Let C be an n by n matrix representing the costs of each of n workers to perform any of n jobs.
The assignment problem is to assign jobs to workers in a way that minimizes the total cost. 
Since each worker can perform only one job and each job can be assigned to only one worker 
the assignments represent an independent set of the matrix C.

One way to generate the optimal set is to create all permutations of the indexes necessary 
to traverse the matrix so that no row and column are used more than once.

http://software.clapper.org/munkres/

22-11-2020

Non-square Cost Matrices
The Munkres algorithm assumes that the cost matrix is square. However, it’s possible to use
a rectangular matrix if you first pad it with 0 values to make it square. 
This module automatically pads rectangular cost matrices to make them square.

Notes:

The module operates on a copy of the caller’s matrix, so any padding will not be
seen by the caller. The cost matrix must be rectangular or square. 
An irregular matrix will not work.
'''

from munkres import Munkres , print_matrix
import numpy as np
import pandas as pd
#--------------------------------
def hungarian(h_matrix, bk_tasks, bk_agents, tasks_arr, agents_arr, all_matrix_dist):
    print('-------------------------------------')
    print('|------ Hungarian Algorithm --------|')
    print('-------------------------------------')

    arr = h_matrix
   
    # Number rows and columns
    l, c = h_matrix.shape
    
    l = bk_agents
    c = bk_tasks
    x_agents = bk_agents
    x_tasks  = bk_tasks  
    
    if l > c:
        width = l
    else:
        width = c
        
    print('Original Matrix - Agents (row):',x_agents,'  Tasks (col):',x_tasks) 

    #------------------------
    # create a list with Agents
    i =0
    agents = []
    while i < l:
        name = 'Agent_'+str(i)
        agents.insert(i,name)
        i = i + 1
    #print(agents)   

    #------------------------------------
    # Obtain Max an min value from matrix
    matrix_arr = np.array(h_matrix)
    
    list_matrix = matrix_arr.tolist()
    square = arr
    
    #print(list_matrix, type(list_matrix))
    
    # Max and Minimum
    list_matrix = matrix_arr.flatten()
    
    max_value, min_value = max_and_min(matrix_arr)
   
    print('Max Value:', max_value,'  Min Value:', min_value)

    #------------------------------------------
    # Insert agents to new lines
    i       = 0
    n_agent = l
    while i < (c - l ):
        name = agents[i]
        agents.insert(n_agent, name)
        n_agent = n_agent + 1 
        i = i + 1
    agents
    #--------------------------------------
    # Transform numpy array to a list
    square_list = np.array(square).tolist()
    square_list

    # - inicialize Algoritm Munkres
    m = Munkres()

    #-------------------------------------
    indexes = m.compute(square_list)

    print_matrix(square_list, msg='Lowest cost through this square  matrix:')
    total = 0
    print('--------------')
    print('Assigned Tasks Square Matrix')
    i = 0
    for row, column in indexes:
        value = square_list[row][column]
        total += value
        print(agents[row], {row}, ' Assigned task:', {column}, '-> Cost:', {value})
        
        #--- Save values to tasks_arr
        # tasks_arr = 0-Task, 1-gent assigned to the task, 2-Cost
        tasks_arr[column,0] = column  # Task number
        tasks_arr[column,1] = row     # Agent assigned to the task
        tasks_arr[column,2] = value   # Cost
        
        #--- save values to agents_arr
        # agents_arr = 0-Agent, 1-Current position, 2-Sum Cost
        agents_arr[row,0] = row     # Agent number
        agents_arr[row,2] = column  # Current position task number
        agents_arr[row,3] = agents_arr[row,2] + value   # Cumulate Cost        
        
    print(f'Total cost Square: {total}')
    x_alg = 'Hungarian'
    x_agent = l
    x_task = c
    indexes

    #---------------------------------------------------------------
    type_assign = 1   # Assign The agent closer A the task consider first assignment position
    type_assign = 2   # Assign The agent is free to do the task
    type_assign = 3   # Assign The agent closer B the task consider last position agent

    if bk_tasks != bk_agents:
        print('-------Execute Process After Hungarian -----------', l, c)       
        tasks_arr = assign_after(all_matrix_dist, tasks_arr, agents_arr, type_assign )
    else:
        print('------ Dont need Process After Hungarian ---------', l, c)
       
    return x_alg, x_agent, x_task, tasks_arr 


In [3]:
#-----------------------------------------------------------------------------
def assign_after(all_matrix_dist, tasks_arr, agents_arr, type_assign ):
    
    # tasks_arr  = 0-Task,  1-Agent assigned to the task, 2-Cost
    # agents_arr = 0-Agent, 1-Number tasks assumed, 2-Current position, 3-Sum Cost
    
    #Number rows and columns
    tl, tc = tasks_arr.shape
    al, tc = agents_arr.shape    
    #------------------------
    print('Tasks assign_after:', tl, al)  
    print('0-Task,  1-Agent assigned to the task, 2-Cost')
    print(tasks_arr)
    #---------------
    print('Agents assign_after:', tl, al) 
    print('0-Agent, 1-Number tasks assumed, 2-Current position, 3-Sum Cost')
    print(agents_arr)
    
    many_nine = 99999999999999999999
    free_tasks = True
    while free_tasks == True:
        n1_line = 0
        # Find Task without costs (Free Task)
        order_cost_tasks_arr = tasks_arr
        agent_cost_tasks_arr = agents_arr
        
        for task in order_cost_tasks_arr:           
            
            if order_cost_tasks_arr[n1_line,2] == 0: # Cost zero Free Task
                #print('Cost 0 for task:', n1_line)
                order_cost_tasks_arr = tasks_arr
                
                # Look for the First free agent (the minimum cost)
                
                cost_agent     = many_nine
                position_agent = many_nine
                
                # Cicle for select the closer agent for this task considering first assigned position
                zcol = 0 
                if type_assign == 1:
                    while zcol < tl: # All tasks
                        if order_cost_tasks_arr[zcol,2] > 0 and n1_line != zcol:
                            a=1
                            #print(' Search closer:', zcol, n1_line, 'Value:', all_matrix_dist[zcol, n1_line],  'Agent Closer:', order_cost_tasks_arr[zcol,2])

                        if all_matrix_dist[zcol, n1_line] <  cost_agent and order_cost_tasks_arr[zcol,2] > 0 and n1_line != zcol:
                            free_agent     = order_cost_tasks_arr[zcol,1]
                            position_agent = order_cost_tasks_arr[zcol,0]
                            cost_agent     = all_matrix_dist[zcol, n1_line] 
                            print('>> ..Closer Agent A:',free_agent, ' Current position:', position_agent, ' Cost:', cost_agent )
                        zcol += 1
                        
                # Cicle for selection the first free agent 
                n2_line = 0
                if type_assign == 2:  
                    agent_cost_tasks_arr = agents_arr
                    for agent in agent_cost_tasks_arr:  # All agents
                        if agent_cost_tasks_arr[n2_line,3] > 0 and agent_cost_tasks_arr[n2_line,3] < cost_agent:
                            free_agent     = agent_cost_tasks_arr[n2_line,0]
                            position_agent = agent_cost_tasks_arr[n2_line,2]
                            cost_agent     = agent_cost_tasks_arr[n2_line,3]
                            print('>> ..Free Agent :', n2_line, free_agent, ' Cost:', cost_agent )
                        n2_line += 1   
                        
                # Cicle for select the closer agent for this task considerind last position
                zcol = 0 
                if type_assign == 3:
                    while zcol < al:   # All agents
                        if agent_cost_tasks_arr[zcol,2] > 0 and n1_line != zcol:
                            a=1
                            #print(' Search closer:', zcol, n1_line, 'Value:', 
                            #      all_matrix_dist[zcol, n1_line],  'Agent Closer:', agent_cost_tasks_arr[zcol,0])
                        
                        zp_task = agent_cost_tasks_arr[zcol,2]
                        if all_matrix_dist[zp_task, n1_line] <  cost_agent and n1_line != zcol:
                            free_agent     = agent_cost_tasks_arr[zcol,0]
                            position_agent = agent_cost_tasks_arr[zcol,2]
                            cost_agent     = all_matrix_dist[zp_task, n1_line] 
                            print('>> ..Closer Agent B:',free_agent, ' Current position:', position_agent, ' Cost:', cost_agent )
                        zcol += 1  
                        
                #cost_agent =  all_matrix_dist[n1_line, position_agent]  
                print('>> ..Free Task:', n1_line,' Assigned to Agent:', free_agent,' > Current Position:', position_agent, '  Cost:', cost_agent)    
                tasks_arr[n1_line,1] = free_agent  # Agent
                tasks_arr[n1_line,2] = cost_agent  # Cost
                #
                agents_arr[free_agent,2] = n1_line     # Change Current task 
                agents_arr[free_agent,3] = agents_arr[free_agent,3] + cost_agent  # Sum the current cost
            n1_line += 1
        free_tasks = False

        #print ('Finish Assighn Free Task -Tasks',  tasks_arr)
        #print ('Finish Assighn Free Task -Agents', agents_arr)
        print('Finished Agents assign_after')
    return tasks_arr

In [4]:
'''
Vickrey Auction

Vickrey auctions are a type of sealed-bid auction where all participants bid for the lot 
at the same time without knowing how their competitors are bidding. As a result, 
the winner of the auction is the bidder with the highest bid. 
However, the winner does not pay the highest bid, but the second-highest bid amount. 
This type of auction encourages the participants to bid exactly how much they value 
the good being sold.

https://corporatefinanceinstitute.com/resources/knowledge/other/vickrey-auction/

http://www.masfoundations.org/mas.pdf

MRobalinho : 4-12-2020
'''

import numpy as np
#  Main start for auction vickrey
def main_auction_vickrey(bid_matrix, x_tasks, x_agents):
    print('-------------------------------------')
    print('|-- Vickrey Auction Algotithm ------|')
    print('-------------------------------------')
    
    # Number Agents and task
    print('Original values have Agents:', x_agents, '   Tasks:', x_tasks)
    print('    >Original Bid Transposed Matrix:')
    print(bid_matrix)
    
    par_matrix = bid_matrix
    #--------- part of matrix
    col1 = 0
    col2 = x_agents   # Columns bk_agent 
    rows = x_agents   # rows bk_task 
    if x_agents < x_tasks:
        print('Take part of the matrix')
        par_matrix = part_matrix(bid_matrix, rows, col1, col2)
        
    par_matrix  =  bid_matrix   # Block the process to partition matrix   
    print(par_matrix) 
    
    #print('    >Transposed Matrix:')
    #print('    ', matrix_t)

    arr = np.array(par_matrix)
    # Number rows and columns
    l, c = arr.shape
    
 #---------------------------------------   
    # Create array with agent score (score_arr)
    # Score_arr = (0=Agent, 1=Number task assumed ,  2= Current position, 3 = Sum Cost)
    score_x = []
    row, col  =  x_agents, 4
    score_x = [[0 for x in range(col)] for y in range(row)] 
    score_arr = np.array(score_x)
    i = 0
    while i < x_agents:
        score_arr[i, 0] = i   # Save agents numbers
        i += 1
    #print('Created array agent score:', score_arr)
#---------------------------------------    
    # Creates a null list containing l(one for each task) lists, 
    # rows = tasks
    # Winners_arr = (0=Task, 1=agent, 2 = bid winner)
    row, col  = x_tasks, 3
    bk_task   = x_tasks  # backup for task number
    bk_agent  = x_agents    # Backup for agent number    
    winners = [[0 for x in range(col)] for y in range(row)] 
    winners_arr = np.array(winners)
    i = 0
    while i < x_tasks:
        winners_arr[i, 0] = i   # Save tasks numbers
        i += 1
    #print('Created array task:', winners_arr)
#---------------------------------------
    # Maximum and minimum value
    max_value, min_value = max_and_min(par_matrix)
    #print('Rows:',l, 'Columns:', c, '  Max:', max_value,'  Min:', min_value)
    
#---------------------------------------
    # Calculate Bid Winners
    winners_arr = calc_bid(par_matrix, winners_arr, score_arr, x_agents, x_tasks)
    
#---------------------------------------      
    type_assign = 1   # Assign The agent closer A the task consider first assignment position
    type_assign = 2   # Assign The agent is free to do the task
    type_assign = 3   # Assign The agent closer B the task consider last position agent

    if x_tasks != x_agents:
        print('-------Execute Process After Vickrey -----------', x_agents, x_tasks)       
        tasks_arr = assign_after(bid_matrix, winners_arr, score_arr, type_assign )
    else:
        print('------ Dont need Process After Vickrey  ---------', x_agents, x_tasks)

#---------------------------------------
    #  call last scores to print
    end_run = 1
    winners_arr, score_arr = create_scores(winners_arr, x_agents, x_tasks, score_arr, end_run)
#---------------------------------------    
    # Calculate total process value
    x_total = calc_total(winners_arr)
    #print('---- Finish Vickrey Auction - Winners ------')
    #print(winners_arr)
    #print('---- Finish Vickrey Auction - Scores ------')
    #print(score_arr)    
    
    return x_total, winners_arr

#-----------------------------------------------------------------
# Calculate the best bids for each task
def calc_bid(par_matrix, winners_arr, score_arr, bk_agent, bk_task):
    
    # Select maximum bid in each task (each row)
    arr = np.array(par_matrix)
    print('Calc winners Bib part of matrix:')
    print(par_matrix)
    
    # Number rows and columns
    t_tasks  = bk_task
    t_agents = bk_agent
    many_nine = 9999999
      
    # Calculate winners
    n_task = 0
    #print('Agents:', t_agents, '  Tasks:', t_tasks)
    for for_task in par_matrix:
        max_bid        = many_nine          # Change to hight (99999) or Low (0)
        last_bid_agent = many_nine
        #print('>> Calc winners - Task number:',n_task)
        n_agent = 0
        #print('>> Search winners-Task:',n_task)            
        while n_agent < bk_agent and n_task < bk_agent: # Only process the number task at same agents
  
            this_bid = for_task[n_agent]  # Bid Value from row/column matriz   
            if score_arr[n_agent, 3] > 0:    # This agent Have tasks (have cost)
                # Penalty (This bid * (Numbers tasks he has + 1))
                this_bid = ( score_arr[n_agent, 1] + 1 )  * this_bid   # Incrase a Time Cost to this Agent(Penalty)
                print('    > Penalty:',n_agent, ' Bid:', for_task[n_agent], 'new >',this_bid)

            #print('    > -Task:',n_task, ' Agent: ', n_agent,'  Bid:', this_bid, ' Max Bid:', max_bid)

            # Find free agents
            i = 0
            have_free_agent = False
            is_free_agent = False
            while i < t_agents:
                #print('score_arr:',i, ' Total agents:', t_agents)
                # score_arr=0-Agent, 1-Number tasks assumed, 2-Current position, 3-Sum Cost
                if score_arr[i, 3] == 0:    # Don't tasks (have no cost)
                    have_free_agent = True 
                if score_arr[i, 3] == 0 and i == n_agent: # this is an free agent
                    is_free_agent = True 
                    #print('    > Agent free:',n_agent)
                i += 1
             
            #have_free_agent = False  # Block processo below
            if have_free_agent == False: # No more free agents
                is_free_agent = True     # Every one can assign the task

            # Select Best bid
            win = False
            if this_bid < max_bid and is_free_agent == True: # Change to hight (99999) or Low (0)
                #print('       ->1 Win:', n_agent, '  Bid:', this_bid)
                older_max_bid  = max_bid       
                last_bid_agent = n_agent      # Agent
                max_bid        = for_task[n_agent]   # Best bid
                win = True           
            n_agent += 1
            
        if last_bid_agent < many_nine:   # Founded a Agent to the task
            #print('       >> Task winner:',n_task, '  Agent: ', last_bid_agent, ' MaxBid:', max_bid)
            #winners_arr[n_task,0] = n_task        # Task
            winners_arr[n_task,1] = last_bid_agent # Agent
            winners_arr[n_task,2] = max_bid        # Value

            # score_arr=0-Agent, 1-Number tasks assumed, 2-Current position, 3-Sum Cost
            score_arr[last_bid_agent, 1] += 1
            score_arr[last_bid_agent, 2] = n_task
            score_arr[last_bid_agent, 3] += max_bid         
        
        n_task += 1

    # print array winners    
    print('Winners - winners_arr :')
    winners_arr
     
    return winners_arr
#------------------------------------------------------
def calc_total(winners_arr):
    print('--- Calculate totals Vickrey auction----')
 
    i  = 0
    x_total = 0
    for task in winners_arr:
        x_total  += winners_arr[i,2]
        i += 1
    
    xt = x_total
    #print('Bid Total value:',x_total)
    #print('Winners Matrix row=Tasks , Col_0 =Task number, Col_1=Agent, Col_2=Bid Value')
    #winners_arr
          
    return x_total

#---------------------------------------------------------------------
# Create the agent scores
def create_scores(winners_arr, x_agent, bk_task, score_arr, end_run):
    
    t_task   = bk_task
    t_agents = x_agent
        
    if end_run == 1:
        print('Initial Create scores_1: score_arr', x_agent, bk_task)
        #print(score_arr)
        #print('Initial Create scores_1: Winners_arr')
        #print(winners_arr)
  
    # Score_arr = columns (0=Agent, 1=Number task assumed ,  2= Current position, 3 = Sum Cost)
    # zero scores for recalculation
    i = 0
    while i < x_agent:
       # score_arr=0-Agent, 1-Number tasks assumed, 2-Current position, 3-Sum Cost
        score_arr[i, 1]  = 0   # Number tasks assumed
        score_arr[i, 3]  = 0   # Sum Cost
        i += 1
   
    # Select maximum bid in each task (each row)
    # Col 1 has the winners agents, for each task
    if end_run == 1:
        print('-------------------------')
        print('WINNERS Vickrey Auction:')
        print('- Agent/ N.Bidden/ Position/Sum Cost-')
    i = 0
    # Winners_arr = columns (0=Task, 1=agent, 2 = bid winner)
    for row in winners_arr:    
        #print(' > Tasks number:',winners_arr_t[0, i], ' Winner:', winners_arr_t[1, i], '=', winners_arr_t[2, i])

        if winners_arr[i, 2] >= 0: # Bid value need to be > 0:
            winner_agent = winners_arr[i ,1]  # Agent number
            if winners_arr[i, 2] > 0:
                # score_arr=0-Agent, 1-Number tasks assumed, 2-Current position, 3-Sum Cost
                score_arr[winner_agent, 1 ] += 1                  # Number times that win
                score_arr[winner_agent, 2 ]  = winners_arr[i, 0]  # Current position
                score_arr[winner_agent, 3 ] += winners_arr[i, 2]  # Sum cost
            if end_run == 1:
                print('  Task > ', i, ' Agent:', winner_agent, '  Bid:', winners_arr[i, 2])
        i += 1
      
        
    # Score pontuation to the agents winners
    if end_run == 1:
        print('Agent scores - score_arr:')
        print('- Agent/Number Wins/Current Position/Cost -')
        print(score_arr)
        
    return winners_arr, score_arr

#----------------------------------------------------------
# Create Bid matriz usin original matriz and insert a % price variation
def create_bid_matrix(b_matrix, par_var):
    #print('Original Matrix for bid:', b_matrix)
    
   # par_var = 50  # % price variation for bid
    var_p = par_var / 100
    print(' > Create Bid Matrix :', par_var , '%')    
    
    bid_matrix = b_matrix
    line = 0
    col = 0 
    for row in bid_matrix: 
        for col in range(len(row)):  
            min_value = int(row[col] - (row[col] * var_p))
            max_value = int(row[col] + (row[col] * var_p))

            #print(min_value, max_value)
            rand_value = random.randint(min_value, max_value)
            if rand_value == 0:
                rand_value = 1    
            #print('  Row >', line, col, 'Act.Value:', row[col], '  Min:', min_value, ' Max:',max_value, 'Random:',rand_value, )
            row[col] = rand_value
            col += 1
        line += 1
    #print('Bid matrix:',bid_matrix)
    
    return bid_matrix
#------------------------------------------------------------
# Define Maximum and Mininum vales
def max_and_min(z_matrix):
    line = 0
    col = 0 
    min_value = 999999999999999
    max_value = 0
    
    for row in z_matrix: 
        for col in range(len(row)):  
            if row[col]  < min_value:
                min_value = row[col]
            if row[col]  > max_value:
                max_value = row[col]
            col += 1
        line += 1
    return max_value, min_value

In [5]:
#----

In [6]:
# X_count - Cicle execution counter
# Par_var - Price variation defined
def global_call(x_agent, x_task, k_matrix, tasks_arr, agents_arr, all_matrix_dist, matrix_dist_start, x_count, par_var):
    
    matrix = k_matrix
    arr    = np.array(matrix)
    x_alg  = 'Hungarian'
    df_h = pd.DataFrame()
    df_a = pd.DataFrame()
    
    print('============= Hungarian algorithm (', x_count,') ======================')
    # Call Hungarian algorithm
    x_alg, x_agent, x_task, tasks_arr = hungarian(arr, x_task, x_agent, tasks_arr, agents_arr, all_matrix_dist)
    
    dfh_tasks = pd.DataFrame({'ID': x_count,'Algorithm': 'Hungarian', 'Task': tasks_arr[:, 0], 'Agent': tasks_arr[:, 1], 'Cost': tasks_arr[:, 2]})
    x_total=dfh_tasks.loc[dfh_tasks['ID'] > 0 ,'Cost'].sum()
    print('--- Tasks assignments Hungarian--- Total Cost:', x_total)
    print('Print Hungarian assignments dfh_tasks:')
    print('0-Task, 1-Agent assigned , 2-Cost')
    print(dfh_tasks)
    
    # Save values
    df_h = pd.DataFrame([([x_alg, x_agent, x_task, x_total ])], columns=['Algoritmo','Agents', 'Tasks', 'Cost'])

    arr = np.array(matrix)
    # Number rows and columns
    l, c = arr.shape

    #------------------------------------------------------------------
    # Call Auction algorithm Vickrey

    # bid_number = num. bidden
    # agent_number = number agents
    # max_value_bid = maximum value for bid

    bid_number = 1
    agent_number = l  # Number agents from the matrix
    #max_value_bid = par_max_value
    tasks = c         # Number of tasks from the matrix

    print('============= Vickrey Auction algorithm (', x_count,') ======================')

    # Transpose
    matrix_arr = np.array(matrix_dist_start)
    matrix_t   = matrix_arr.transpose()
    x_alg  = 'Vickrey'
    
    # Create Bid Matrix using original matrix (matrix, % bid price variation)
    bid_matrix = matrix_t
    bid_matrix = create_bid_matrix(matrix_t, par_var)
    
    # Call auction algorithm
    x_total, winners_arr = main_auction_vickrey(bid_matrix, x_task, x_agent)
    #print('--- Finish Vickrey - winner_arr ---')
    #print(winners_arr)
    
    dfa_tasks = pd.DataFrame({'ID': x_count,'Algorithm': 'Vickrey', 'Task': winners_arr[:, 0], 'Agent': winners_arr[:, 1], 'Cost': winners_arr[:, 2]})
    x_total=dfa_tasks.loc[dfa_tasks['ID'] > 0 ,'Cost'].sum()
    print('--- Tasks assignments Vickrey--- Total Cost:', x_total)
    print('Print Vickrey assignments dfa_tasks:')
    print('0-Task, 1-Agent assigned , 2-Cost')    
    print(dfa_tasks)
        
    # registo de valores no dataframe
    x_alg   = 'Vickrey Auction'
    x_agent = agent_number
    x_task  = tasks
    df_a = pd.DataFrame([([x_alg, x_agent, x_task, x_total ])], columns=['Algoritmo','Agents', 'Tasks', 'Cost'])

    return df_h, df_a, dfa_tasks     


#--------------------------------------------------------------------
def main_work(par_number_of_executions, par_var, par_min_cost, par_max_cost, par_max_agents, par_max_tasks, par_tkinter):
    
    # par_tkinter- Label for par_tkinter screen
    x_dfh = pd.DataFrame()
    x_dfa = pd.DataFrame()
    dfx   = pd.DataFrame()
    par_i = 1   # To print with Tkinter
    # Print date and time     
    now = datetime.datetime.now()
    print (" ")
    print ("Process start at date and time : ", now.strftime("%Y-%m-%d %H:%M:%S"))
    
    while par_i <= par_number_of_executions:   # do n times
        if par_tkinter == 1:
            # Label for par_tkinter screen
            message = 'Execution number:'+str(par_i)
            e_disp = Label(window, text = message,anchor='w').grid(row = 23, column = 1)
        
        print('========================================================')
        print('=====>> Start Global Cicle :', par_i)
        print('========================================================')
        # Random number for Task and agent
        x_agent = random.randint(2, par_max_agents)   # Minimum 2 Agents
        x_task  = random.randint(2, par_max_tasks)    # Minimum 2 tasks
        print('Random Task:', x_task,'  Agent:', x_agent)
              
        if x_agent > x_task:
            x_agent = x_task
        
        #x_agent = 4    # Comment here - force number agents
        #x_task = 6     # Comment here - forne number tasks
        
        bk_agent = x_agent 
        bk_task  = x_task

        print('Assumed Task:', x_task,'  Agent:', x_agent)

        # Random tasks and agents Matrix
        k_matrix = np.random.randint(par_min_cost, par_max_cost, size=(x_agent, x_task))
        #k_matrix
        
        #--- Random arrays for distances, task and agents
        tasks_arr, agents_arr, all_matrix_dist, par_arr, matrix_dist_start = random_arrays(bk_task, bk_agent, par_min_cost, par_max_cost)
        #k_matrix = par_arr
        
        # Create matrix wid Bids using original matrix
        # ---  Bid Matrix
        matrix_arr = matrix_dist_start
  
        # Global Call
        # par_i - signal to print using Tkinter
        # par_var - price variation defined
        x_dfh, x_dfa, df_tasks = global_call(x_agent, x_task, k_matrix, tasks_arr, agents_arr, all_matrix_dist, matrix_dist_start, par_i, par_var )
        
       
        #print('Tasks',  tasks_arr)
        #print('Agents', agents_arr)
        # Write results in Pandas DataFrame
        x_dfh.loc[:,'ID'] = par_i
        x_dfa.loc[:,'ID'] = par_i

        hx = x_dfh.iloc[0]['Cost']
        ax = x_dfa.iloc[0]['Cost']
        
        if par_tkinter == 1:
            # Label for par_tkinter screen
            message1 = 'Hungarian algorithm Cost = '+str(hx)
            message2 = 'Vickrey auction Cost = '+str(ax)
            message3 = 'Process Finished, look excel file > UC_MR_Mywork.xlsx'
            e_disp = Label(window, text = message1,anchor='w').grid(row = 24, column = 1)
            e_disp = Label(window, text = message2,anchor='w').grid(row = 25, column = 1)
            e_disp = Label(window, text = message3,anchor='w').grid(row = 26, column = 1)
        
        if hx < ax:                         # Winner Hungarian (Less Cost) 
            x_dfh.loc[:,'Winner_Hungarian'] = 1  
            x_dfh.loc[:,'Winner_Vickrey']   = 0 

            x_dfa.loc[:,'Winner_Hungarian'] = 1  
            x_dfa.loc[:,'Winner_Vickrey']   = 0  
        else:                                # Winner Vickrey (Less Cost) 
            x_dfh.loc[:,'Winner_Hungarian'] = 0 
            x_dfh.loc[:,'Winner_Vickrey']   = 1 

            x_dfa.loc[:,'Winner_Hungarian'] = 0  
            x_dfa.loc[:,'Winner_Vickrey']   = 1          

        dfx = dfx.append(pd.DataFrame(x_dfh),ignore_index=True)
        dfx = dfx.append(pd.DataFrame(x_dfa),ignore_index=True)

        dfx.index = (dfx.index + 1) % len(dfx)
        dfx = dfx.sort_index()
        del x_dfh
        del x_dfa

        par_i += 1

    dfx = dfx.sort_values('ID') 
    
    # Print date and time     
    now = datetime.datetime.now()
    print (" ")
    print ("Process finished at date and time : ", now.strftime("%Y-%m-%d %H:%M:%S"))
   
    # Write results in Excel file 
    print('Finished, look excel file > UC_MR_Mywork.xlsx')
    dfx.to_excel(r'UC_MR_Mywork.xlsx', index = False)
      
    return df_tasks 

#-----------------------------------------------
# select part of an matrx
def part_matrix( ex, row, col1, col2):    
    vx = ex[:row, col1:col2]
    return vx

#--------------------------------------------------------
# Simetric
def symmetrize(a):
    """
    Return a symmetrized version of NumPy array a.

    Values 0 are replaced by the array value at the symmetric
    position (with respect to the diagonal), i.e. if a_ij = 0,
    then the returned array a' is such that a'_ij = a_ji.

    Diagonal values are left untouched.

    a -- square NumPy array, such that a_ij = 0 or a_ji = 0, 
    for i != j.
    """
    print('--- all task distances---')
    matrix_dist = a + a.T - np.diag(a.diagonal())
    return matrix_dist

#-----------------------------------------------------------
# Define Maximum and Mininum vales
# All Agents start at the same position, so with the same cost (distance)
def start_initial_task(matrix_dist_start, initial_task):
    line = 0
    col = 0
    x_agent, x_task = matrix_dist_start.shape
    print('--- Matrix task distances for start position :', initial_task, '--- Agents:', x_agent,' tasks:', x_task)
    for row in matrix_dist_start: 
        for col in range(len(row)): 
            if line != initial_task:
                row[col]  = matrix_dist_start[initial_task,col]                
            col += 1
        line += 1
    return matrix_dist_start

#--------------------------------------------------------------
def random_arrays(bk_task, bk_agent, par_min_cost, par_max_cost):
    #-------- Create Distances Matrix
    print('--- Call Random arrays --------------')
    # Random tasks and agents Matrix
    matrix_dist_x = np.random.randint(par_min_cost, par_max_cost, size=(bk_task, bk_task))
    #print('Original Matrix dist:',matrix_dist_x)
    all_matrix_dist = symmetrize(matrix_dist_x)
    print('All Distances:')
    print(all_matrix_dist)

    #-------------------------------------
    # Distances from Start Task
    initial_task = 0
    matrix_dist_start = start_initial_task(all_matrix_dist, initial_task)
    print('--- Matrix start position', initial_task)
    print(matrix_dist_start)
    
    #--------- part of matrix
    par_arr = matrix_dist_start
    col1 = 0
    col2 = bk_agent
    rows = bk_agent
    if bk_agent < bk_task:
        #print('Take part of the matrix')
        par_arr = part_matrix(matrix_dist_start, rows, col1, col2)
    else:
        par_arr = matrix_dist_start
    #print('Part Matrix finish:')
    #print(par_arr)
   
    #------------------------------
    # Create control array tasks
    # Creates a null list containing l(one for each task) lists, 
    # each of 3 items(4 informations), all set to 0
    # rows = tasks
    # tasks_arr= (0=Task, 1=agent, 2 = cost winner)
    row, col  = bk_task, 3; 
    m_tasks = [[0 for x in range(col)] for y in range(row)] 
    tasks_arr = np.array(m_tasks)
    n_task = 0
    for task in tasks_arr:
        tasks_arr[n_task,0] = n_task # put Task number in col 0
        n_task += 1
    #print('Array Task_arr:', tasks_arr) 

    #------------------------------
    # Create control array tasks
    # Creates a null list containing l(one for each agent) lists, 
    # each of 3 items(4 informations), all set to 0
    # rows = tasks
    # agents_arr= (0=Agent, 1- Number assumed task, 2=Position(task), 3 = cost)
    row, col  = bk_agent, 4; 
    m_agents = [[0 for x in range(col)] for y in range(row)] 
    agents_arr = np.array(m_agents)
    n_agent = 0
    for agent in agents_arr:
        agents_arr[n_agent,0] = n_agent  # put Agents number in col 0
        n_agent += 1    
    #print('Array Agent_arr:', agents_arr)

    return tasks_arr, agents_arr, all_matrix_dist, par_arr, matrix_dist_start

In [7]:
#------------------

In [ ]:
# http://www.pythonlake.com/tkinterlabelgrid
#--------------------------------------
# Screen to optimizer algorithm Hungarian and Vickrey
#-------------------------------------
from tkinter import messagebox
from tkinter import *
import tkinter as tk

def print_parameters():
    # get values from screen
    p_v0 = float(e0.get())
    p_v1 = float(e1.get())
    p_v2 = float(e2.get())
    p_v3 = float(e3.get())
    p_v4 = float(e4.get())
    p_v5 = float(e5.get())
    #---------------
    print('----  Parameters imputed------')
    print('Number of executions:', p_v0)
    print('Bid price variation :', p_v1)
    print('Min Cost for matrix :', p_v2)
    print('Max Cost for matrix :', p_v3)
    print('Maximum  Agents     :', p_v4)
    print('Maximum  Tasks      :', p_v5)
    print('---------------------------')
  
    par_tkinter = 1   # Executed with tkinter
    
    df_tasks = main_work(p_v0, p_v1, p_v2, p_v3, p_v4, p_v5 , par_tkinter)
    
    message = 'Process done ' +str(p_v0), ' times'
    e_disp = Label(window, text = message,anchor='w').grid(row = 22, column = 1)
    
    return

#------------------------------------------------

message = ''
# Window and title
window = Tk()
window.title("Welcome to Optimizer algorithms app")
window.geometry('800x400')
#-----

# Label
lbl = Label(window, text=" Execution for algorithms Hungarian and Vickrey Auction",font=("Arial Bold", 15))
lbl.grid(row=1, column=1 )

# Label line
lbl = Label(window, text=" __________________________________________________________",font=("Arial Bold", 15))
lbl.grid(row=2, column=1 )
# Label line
lbl = Label(window, text=" ______________",font=("Arial Bold", 15))
lbl.grid(row=2, column=2 )
      
# Button
btn = Button(window, text="Click Me", bg="blue", fg="white", font=("Arial Bold", 10), command=print_parameters)
btn.grid(column=1, row=10)
  
#-- First line 
v0 = IntVar()
v0.set(5)
lbl0 = Label(window, text = "Number of executions:",anchor='w',font=("Arial Bold", 10)).grid(row = 3, column = 1)
e0 = Entry(window , text = v0) 
e0.grid(row = 3, column = 2, sticky="w")

#--
v1 = IntVar()
v1.set(25)
lbl1 = Label(window, text = "Bid price variation :",relief=RIDGE,anchor='w').grid(row = 4, column = 1)
e1 = Entry(window, text = v1) 
e1.grid(row = 4, column = 2, sticky="w")

#--
v2 = IntVar()
v2.set(1)
lbl2 = Label(window, text = "Min Cost for matrix :",anchor='w').grid(row = 5, column = 1)
e2 = Entry(window, text = v2)
e2.grid(row = 5, column = 2, sticky="w")

#--
v3 = IntVar()
v3.set(100)
lbl3 = Label(window, text = "Max Cost for matrix :",anchor='w').grid(row = 6, column = 1)
e3 = tk.Entry(window, text = v3) 
e3.grid(row = 6, column = 2, sticky="w")

#--
v4 = IntVar()
v4.set(10)
lbl4 = Label(window, text = "Maximum  Agents     :",anchor='w').grid(row = 7, column = 1)
e4 = tk.Entry(window, text = v4) 
e4.grid(row = 7, column = 2, sticky="w")

#--
v5 = IntVar()
v5.set(10)
lbl4 = Label(window, text = "Maximum Tasks      :",anchor='w').grid(row = 8, column = 1)
e5 = tk.Entry(window, text = v5) 
e5.grid(row = 8, column = 2, sticky="w")
#--

lbl = Label(window, text=" __________________________________________________________",font=("Arial Bold", 15)).grid(row=20, column=1)
e_disp = Label(window, text = message,anchor='w').grid(row = 21, column = 1)


window.mainloop()

----  Parameters imputed------
Number of executions: 5.0
Bid price variation : 25.0
Min Cost for matrix : 1.0
Max Cost for matrix : 100.0
Maximum  Agents     : 10.0
Maximum  Tasks      : 10.0
---------------------------
 
Process start at date and time :  2021-01-09 15:06:25
=====>> Start Global Cicle : 1
Random Task: 8   Agent: 8
Assumed Task: 8   Agent: 8
--- Call Random arrays --------------
--- all task distances---
All Distances:
[[ 51 172 121  75 110 140 181 152]
 [172   7 101  63 156  68 105 140]
 [121 101   9  92 164 144  77  98]
 [ 75  63  92  73 104  67 160 117]
 [110 156 164 104  71 114 106 136]
 [140  68 144  67 114  81  85 155]
 [181 105  77 160 106  85  45  51]
 [152 140  98 117 136 155  51  39]]
--- Matrix task distances for start position : 0 --- Agents: 8  tasks: 8
--- Matrix start position 0
[[ 51 172 121  75 110 140 181 152]
 [ 51 172 121  75 110 140 181 152]
 [ 51 172 121  75 110 140 181 152]
 [ 51 172 121  75 110 140 181 152]
 [ 51 172 121  75 110 140 181 152]
 [ 5

============= Vickrey Auction algorithm ( 4 ) ======================
 > Create Bid Matrix : 25.0 %
-------------------------------------
|-- Vickrey Auction Algotithm ------|
-------------------------------------
Original values have Agents: 4    Tasks: 4
    >Original Bid Transposed Matrix:
[[ 14  13  12  11]
 [112 108 147 117]
 [143 128 157 182]
 [100 112 124  99]]
[[ 14  13  12  11]
 [112 108 147 117]
 [143 128 157 182]
 [100 112 124  99]]
Calc winners Bib part of matrix:
[[ 14  13  12  11]
 [112 108 147 117]
 [143 128 157 182]
 [100 112 124  99]]
    > Penalty: 3  Bid: 117 new > 234
    > Penalty: 1  Bid: 128 new > 256
    > Penalty: 3  Bid: 182 new > 364
    > Penalty: 0  Bid: 100 new > 200
    > Penalty: 1  Bid: 112 new > 224
    > Penalty: 3  Bid: 99 new > 198
Winners - winners_arr :
------ Dont need Process After Vickrey  --------- 4 4
Initial Create scores_1: score_arr 4 4
-------------------------
WINNERS Vickrey Auction:
- Agent/ N.Bidden/ Position/Sum Cost-
  Task >  0  Age

Original Matrix - Agents (row): 4   Tasks (col): 5
Max Value: 96   Min Value: 13
Lowest cost through this square  matrix:
[36, 43, 26, 51, 96]
[78, 71, 52, 44, 73]
[41, 80, 28, 57, 13]
[76, 90, 39, 55, 83]
--------------
Assigned Tasks Square Matrix
Agent_0 {0}  Assigned task: {0} -> Cost: {36}
Agent_1 {1}  Assigned task: {3} -> Cost: {44}
Agent_2 {2}  Assigned task: {4} -> Cost: {13}
Agent_3 {3}  Assigned task: {2} -> Cost: {39}
Total cost Square: 132
-------Execute Process After Hungarian ----------- 4 5
Tasks assign_after: 5 4
0-Task,  1-Agent assigned to the task, 2-Cost
[[ 0  0 36]
 [ 1  0  0]
 [ 2  3 39]
 [ 3  1 44]
 [ 4  2 13]]
Agents assign_after: 5 4
0-Agent, 1-Number tasks assumed, 2-Current position, 3-Sum Cost
[[ 0  0  0 36]
 [ 1  0  3 47]
 [ 2  0  4 17]
 [ 3  0  2 41]]
>> ..Closer Agent B: 0  Current position: 0  Cost: 90
>> ..Free Task: 1  Assigned to Agent: 0  > Current Position: 0   Cost: 90
Finished Agents assign_after
--- Tasks assignments Hungarian--- Total Cost: 222

10   6  Hungarian    10      7     1
============= Vickrey Auction algorithm ( 6 ) ======================
 > Create Bid Matrix : 25.0 %
-------------------------------------
|-- Vickrey Auction Algotithm ------|
-------------------------------------
Original values have Agents: 11    Tasks: 11
    >Original Bid Transposed Matrix:
[[ 54  59  59  68  58  54  53  46  60  62  76]
 [ 55  57  77  70  64  61  72  56  77  66  52]
 [113 118 149 114 161 129 106 123 169 145 168]
 [ 94 136 113 134 152 131 147 142 148 149 131]
 [ 91  91  76  80  86 101 106 100 104 114  89]
 [105 125 100 122 114  80  85  86 114  94  76]
 [156 147 152 154 136 127 143 119 152 156 160]
 [129 142 133 100 129 112 118 134 138 118 140]
 [127 100 133 111  95  94 127 119  97 104 101]
 [111 102 108  88  80 107  88  77 109 106  91]
 [ 83  69  58  82  64  80  74  54  74  83  59]]
[[ 54  59  59  68  58  54  53  46  60  62  76]
 [ 55  57  77  70  64  61  72  56  77  66  52]
 [113 118 149 114 161 129 106 123 169 145 168]
 [ 94 136

 [106 129 141 105 110 155  99 142 153 118 112  99 100 138 135]]
Take part of the matrix
[[ 77  88  75  62  91  90  83  74  80  71  83  85  62  82  86]
 [ 37  33  40  27  36  33  30  38  25  39  37  40  33  33  40]
 [128 106 141 109 129 148 128 152  97 105  96 106 134 143 125]
 [ 74  74  55  69  53  72  64  61  83  69  74  58  60  81  79]
 [113 111 114  93 112 121 106  83 111  91  83  99 102  80  94]
 [112 121 136 134 147 142 137 105 118 129  91 120 127  92  93]
 [ 81 118  82 120 107  75 109  80 120 103 110  95 114  94  92]
 [ 35  41  37  41  35  36  40  38  37  28  36  43  36  42  34]
 [160 164 153 138 165 201 185 160 162 170 145 148 127 169 127]
 [ 79  72  76  97  94  99  83  96  87  92  73  93  91  71 107]
 [161 195 186 168 162 225 143 198 157 151 169 171 199 152 156]
 [ 99 103  71  66  93  77  73  73  88  73  62  66  77  97  83]
 [ 79  74 101  93  68  97 102  88  73  69  95  79 112 101  94]
 [ 43  46  46  38  39  49  34  38  45  40  50  53  44  45  35]
 [106 129 141 105 110 155  99 

...

0-Task,  1-Agent assigned to the task, 2-Cost
[[ 0  0  0]
 [ 1  8 17]
 [ 2  0  9]
 [ 3  7  5]
 [ 4  0  0]
 [ 5  4 11]
 [ 6  2 10]
 [ 7  5  6]
 [ 8  0  0]
 [ 9  0  0]
 [10  1  2]
 [11  6  5]
 [12  9  9]
 [13  0  0]
 [14  3  9]]
Agents assign_after: 15 10
0-Agent, 1-Number tasks assumed, 2-Current position, 3-Sum Cost
[[ 0  0  2 11]
 [ 1  0 10 12]
 [ 2  0  6 16]
 [ 3  0 14 23]
 [ 4  0  5 16]
 [ 5  0  7 13]
 [ 6  0 11 16]
 [ 7  0  3  8]
 [ 8  0  1 18]
 [ 9  0 12 21]]
>> ..Closer Agent B: 1  Current position: 10  Cost: 21
>> ..Free Task: 0  Assigned to Agent: 1  > Current Position: 10   Cost: 21
>> ..Closer Agent B: 0  Current position: 2  Cost: 24
>> ..Free Task: 4  Assigned to Agent: 0  > Current Position: 2   Cost: 24
>> ..Closer Agent B: 0  Current position: 4  Cost: 97
>> ..Free Task: 8  Assigned to Agent: 0  > Current Position: 4   Cost: 97
>> ..Closer Agent B: 0  Current position: 8  Cost: 59
>> ..Free Task: 9  Assigned to Agent: 0  > Current Position: 8   Cost: 59
>> ..Closer Agent

Lowest cost through this square  matrix:
[75, 30, 37, 34, 49, 37, 49,  8, 49, 72,  3, 35, 59, 76]
[67, 23, 93, 55, 13, 39, 68, 54, 89, 24,  7, 16, 13, 23]
[98,  6, 44, 36, 54, 61, 46, 89, 77, 86, 96, 95,  8, 79]
[99, 10, 55, 34, 33, 75, 30, 96, 48, 15, 67, 28, 99, 54]
[44, 24, 18, 98, 75, 48, 69, 77, 27,  7, 74, 63, 43, 26]
[61, 34, 89, 68, 21, 46,  6, 81, 98, 28, 59, 57, 63, 81]
[10,  3, 20, 51, 72, 36, 12, 32, 43, 37, 31, 25, 80, 61]
[13, 27, 13, 70, 20,  6, 14, 10, 45, 32, 81, 75,  6, 97]
[53, 28, 46, 35, 56, 98, 85, 86, 58, 25, 45, 25, 97, 35]
[93, 48, 69, 27, 13, 48, 93, 48,  8, 76,  6, 35, 68, 43]
[21, 71, 45, 10, 67,  9, 49, 21, 82, 97, 79, 80, 26, 34]
[93,  8, 69, 80, 91, 21, 98, 31, 26, 29, 76, 46, 48, 19]
[83, 87, 89, 29, 82, 57, 14, 91, 56, 64, 23,  7, 67,  7]
--------------
Assigned Tasks Square Matrix
Agent_0 {0}  Assigned task: {7} -> Cost: {8}
Agent_1 {1}  Assigned task: {10} -> Cost: {7}
Agent_2 {2}  Assigned task: {12} -> Cost: {8}
Agent_3 {3}  Assigned task: {9} -> Co

--- Tasks assignments Vickrey--- Total Cost: 1227
Print Vickrey assignments dfa_tasks:
0-Task, 1-Agent assigned , 2-Cost
    ID Algorithm  Task  Agent  Cost
0   97   Vickrey     0      9    51
1   97   Vickrey     1      2    62
2   97   Vickrey     2      7    55
3   97   Vickrey     3      4    74
4   97   Vickrey     4      3    76
5   97   Vickrey     5      6   111
6   97   Vickrey     6      5    61
7   97   Vickrey     7      1   126
8   97   Vickrey     8      8   139
9   97   Vickrey     9     10    71
10  97   Vickrey    10      0    65
11  97   Vickrey    11     11   121
12  97   Vickrey    12     12   154
13  97   Vickrey    13      9    61
=====>> Start Global Cicle : 98
Random Task: 2   Agent: 20
Assumed Task: 2   Agent: 2
--- Call Random arrays --------------
--- all task distances---
All Distances:
[[ 5 49]
 [49 31]]
--- Matrix task distances for start position : 0 --- Agents: 2  tasks: 2
--- Matrix start position 0
[[ 5 49]
 [ 5 49]]
============= Hungarian algorithm (

>> ..Closer Agent B: 0  Current position: 11  Cost: 66
>> ..Closer Agent B: 1  Current position: 12  Cost: 59
>> ..Free Task: 13  Assigned to Agent: 1  > Current Position: 12   Cost: 59
>> ..Closer Agent B: 0  Current position: 11  Cost: 102
>> ..Free Task: 14  Assigned to Agent: 0  > Current Position: 11   Cost: 102
>> ..Closer Agent B: 0  Current position: 14  Cost: 64
>> ..Free Task: 15  Assigned to Agent: 0  > Current Position: 14   Cost: 64
>> ..Closer Agent B: 0  Current position: 15  Cost: 93
>> ..Free Task: 16  Assigned to Agent: 0  > Current Position: 15   Cost: 93
>> ..Closer Agent B: 0  Current position: 16  Cost: 59
>> ..Free Task: 17  Assigned to Agent: 0  > Current Position: 16   Cost: 59
Finished Agents assign_after
Initial Create scores_1: score_arr 5 18
-------------------------
WINNERS Vickrey Auction:
- Agent/ N.Bidden/ Position/Sum Cost-
  Task >  0  Agent: 4   Bid: 20
  Task >  1  Agent: 0   Bid: 82
  Task >  2  Agent: 3   Bid: 77
  Task >  3  Agent: 1   Bid: 104
 

    > Penalty: 1  Bid: 67 new > 134
Winners - winners_arr :
-------Execute Process After Vickrey ----------- 2 12
Tasks assign_after: 12 2
0-Task,  1-Agent assigned to the task, 2-Cost
[[ 0  1 17]
 [ 1  0 65]
 [ 2  0  0]
 [ 3  0  0]
 [ 4  0  0]
 [ 5  0  0]
 [ 6  0  0]
 [ 7  0  0]
 [ 8  0  0]
 [ 9  0  0]
 [10  0  0]
 [11  0  0]]
Agents assign_after: 12 2
0-Agent, 1-Number tasks assumed, 2-Current position, 3-Sum Cost
[[ 0  1  1 65]
 [ 1  1  0 17]]
>> ..Closer Agent B: 0  Current position: 1  Cost: 87
>> ..Closer Agent B: 1  Current position: 0  Cost: 15
>> ..Free Task: 2  Assigned to Agent: 1  > Current Position: 0   Cost: 15
>> ..Closer Agent B: 0  Current position: 1  Cost: 90
>> ..Free Task: 3  Assigned to Agent: 0  > Current Position: 1   Cost: 90
>> ..Closer Agent B: 0  Current position: 3  Cost: 83
>> ..Free Task: 4  Assigned to Agent: 0  > Current Position: 3   Cost: 83
>> ..Closer Agent B: 0  Current position: 4  Cost: 72
>> ..Free Task: 5  Assigned to Agent: 0  > Current Positi

In [10]:
#-------------------

In [80]:
#=======================================
#    Manual Start Process without screen
#=======================================

# --------------  Define parameters to execution
max_value = 999
par_number_of_executions = 1  
par_var = 25         # % price variation for bid
par_min_cost = 1     # cost minimum to generate random matrix cost
par_max_cost = 100   # cost maximum to generate random matrix cost
par_max_agents = 10  # maximum agents
par_max_tasks  = 10  # maximum tasks
# ----------------------------------------------
print('----  Run Parameters ------')
print('Number of executions:', par_number_of_executions)
print('Bid price variation :', par_var)
print('Min Cost for matrix :', par_min_cost)
print('Max Cost for matrix :', par_max_cost)
print('Maximum  Agents     :', par_max_agents)
print('Maximum  Tasks      :', par_max_tasks)
print('---------------------------')

#--------------------------
par_tkinter = 0

df_tasks = main_work(par_number_of_executions, par_var, par_min_cost, 
                     par_max_cost, par_max_agents, par_max_tasks, par_tkinter )

#-------------------------------------------

----  Run Parameters ------
Number of executions: 1
Bid price variation : 25
Min Cost for matrix : 1
Max Cost for matrix : 100
Maximum  Agents     : 10
Maximum  Tasks      : 10
---------------------------
 
Process start at date and time :  2020-12-25 14:08:10
=====>> Start Global Cicle : 1
Random Task: 9   Agent: 2
Assumed Task: 6   Agent: 4
--- Call Random arrays --------------
--- all task distances---
All Distances:
[[ 28 114 120 110 174 133]
 [114  26  90  34 135 161]
 [120  90  71 192   8 170]
 [110  34 192  56 112  68]
 [174 135   8 112  31  72]
 [133 161 170  68  72  28]]
--- Matrix task distances for start position : 0 --- Agents: 6  tasks: 6
--- Matrix start position 0
[[ 28 114 120 110 174 133]
 [ 28 114 120 110 174 133]
 [ 28 114 120 110 174 133]
 [ 28 114 120 110 174 133]
 [ 28 114 120 110 174 133]
 [ 28 114 120 110 174 133]]
============= Hungarian algorithm ( 1 ) ======================
-------------------------------------
|------ Hungarian Algorithm --------|
----------

In [40]:
#------------

In [50]:
# --------------------------------
# TESTE area Vickrey
#---------------------------------

bk_task = 6
bk_agent = 4
par_min_cost = 1
par_max_cost = 20
par_var = 20
#--- Random arrays for distances, task and agents
tasks_arr, agents_arr, all_matrix_dist, par_arr, matrix_dist_start = random_arrays(bk_task, bk_agent, par_min_cost, par_max_cost)
      
# ---  Bid Matrix
matrix_arr = matrix_dist_start
arr = create_bid_matrix(matrix_arr, par_var)

# Transpose
b_matrix = arr.transpose()
print('--- Finish random--')
b_matrix


--- Call Random arrays --------------
--- all task distances---
All Distances:
[[ 7 23 10 19 20 24]
 [23 12 18 24  3  6]
 [10 18  1 33 32 14]
 [19 24 33 14 12 14]
 [20  3 32 12 11 22]
 [24  6 14 14 22  7]]
--- Matrix task distances for start position : 0 --- Agents: 6  tasks: 6
--- Matrix start position 0
[[ 7 23 10 19 20 24]
 [ 7 23 10 19 20 24]
 [ 7 23 10 19 20 24]
 [ 7 23 10 19 20 24]
 [ 7 23 10 19 20 24]
 [ 7 23 10 19 20 24]]
 > Create Bid Matrix : 20 %
--- Finish random--


array([[ 7,  5,  6,  7,  8,  8],
       [23, 25, 26, 25, 27, 26],
       [ 8,  8, 11, 11,  9, 10],
       [16, 18, 19, 19, 15, 19],
       [24, 17, 16, 17, 17, 18],
       [21, 22, 27, 23, 20, 25]])

In [51]:
# Block for tests     
main_auction_vickrey(b_matrix, bk_task, bk_agent)

-------------------------------------
|-- Vickrey Auction Algotithm ------|
-------------------------------------
Original values have Agents: 4    Tasks: 6
    >Original Bid Transposed Matrix:
[[ 7  5  6  7  8  8]
 [23 25 26 25 27 26]
 [ 8  8 11 11  9 10]
 [16 18 19 19 15 19]
 [24 17 16 17 17 18]
 [21 22 27 23 20 25]]
Take part of the matrix
[[ 7  5  6  7  8  8]
 [23 25 26 25 27 26]
 [ 8  8 11 11  9 10]
 [16 18 19 19 15 19]
 [24 17 16 17 17 18]
 [21 22 27 23 20 25]]
Calc winners Bib part of matrix:
[[ 7  5  6  7  8  8]
 [23 25 26 25 27 26]
 [ 8  8 11 11  9 10]
 [16 18 19 19 15 19]
 [24 17 16 17 17 18]
 [21 22 27 23 20 25]]
    > Penalty: 1  Bid: 25 new > 50
    > Penalty: 0  Bid: 8 new > 16
    > Penalty: 1  Bid: 8 new > 16
    > Penalty: 0  Bid: 16 new > 32
    > Penalty: 1  Bid: 18 new > 36
    > Penalty: 2  Bid: 19 new > 38
Winners - winners_arr :
-------Execute Process After Vickrey ----------- 4 6
Tasks assign_after: 6 4
0-Task,  1-Agent assigned to the task, 2-Cost
[[ 0  1  5]
 

(76,
 array([[ 0,  1,  5],
        [ 1,  0, 23],
        [ 2,  2, 11],
        [ 3,  3, 19],
        [ 4,  1,  8],
        [ 5,  2, 10]]))

In [52]:
#  ----------------------------------------------------------------

In [45]:
#'''
# ---- Block only for TEST ----
#
# Rows - Agents
# Columns = tasks
import numpy as np

#----------------------------
x_count = 1
matrix = []
matrix = [[1, 9, 6, 4, 1, 9],
[8, 4, 6, 4, 4, 5],
[7, 9, 9, 5, 3, 4],      
[2, 4, 5, 1, 7, 6],
[3, 5, 5, 3, 3, 2]]

# Transform list to numpy array
matrix_arr = np.array(matrix)
print('Original matrix')
print(matrix_arr)

# Number rows and columns
bk_agent, bk_task = matrix_arr.shape
print('Agents:', bk_agent,'  Tasks:', bk_task)
par_min_cost = 1
par_max_cost = 10

#create_scores(winners_arr, x_agents, bk_task, 1)


#------- TEST AUCTION ----------------------
#l=0    
#max_value = max([max(l) for l in matrix_arr])
#min_value = min([min(l) for l in matrix_arr])
#print('Max:',max_value , '   Min:', min_value)
    
#x_total = main_auction_vickrey(matrix_arr, bk_task, bk_agent)

#'''
'''
#----------------------------------
# TEST Normalization - Some agents with many task and others with none
#matrix_t = matrix_arr.transpose()
print('TESTE NORMALIZATION')    
score =  [[3, 1, 0, 0, 1]]
score_arr = np.array(score)
print('Scores:',score_arr)

winners =[[ 0 , 4 ,21],
          [ 6 , 0 ,1]]
winners_arr = np.array(winners)
print('Winners:')
winners_arr

# Transpose
matrix_arr = np.array(matrix)
matrix_t = matrix_arr.transpose()
print('Matrix_transposed:')
matrix_t

winners_arr, score_arr  = verify_normalization(matrix_t, score_arr, winners_arr)
#----------------------------------------

print('--- agents -----')
#print(x_matrix_t)
print('----------------')
'''

Original matrix
[[1 9 6 4 1 9]
 [8 4 6 4 4 5]
 [7 9 9 5 3 4]
 [2 4 5 1 7 6]
 [3 5 5 3 3 2]]
Agents: 5   Tasks: 6


"\n#----------------------------------\n# TEST Normalization - Some agents with many task and others with none\n#matrix_t = matrix_arr.transpose()\nprint('TESTE NORMALIZATION')    \nscore =  [[3, 1, 0, 0, 1]]\nscore_arr = np.array(score)\nprint('Scores:',score_arr)\n\nwinners =[[ 0 , 4 ,21],\n          [ 6 , 0 ,1]]\nwinners_arr = np.array(winners)\nprint('Winners:')\nwinners_arr\n\n# Transpose\nmatrix_arr = np.array(matrix)\nmatrix_t = matrix_arr.transpose()\nprint('Matrix_transposed:')\nmatrix_t\n\nwinners_arr, score_arr  = verify_normalization(matrix_t, score_arr, winners_arr)\n#----------------------------------------\n\nprint('--- agents -----')\n#print(x_matrix_t)\nprint('----------------')\n"